<a href="https://colab.research.google.com/github/INSVICKY/Machine_learning_jovian/blob/main/Vivek_python_sklearn_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
##Downloading the Data
##To begin, let's download the data using the urlretrieve function from urllib.request.
medical_charges_url = 'https://raw.githubusercontent.com/JovianML/opendatasets/master/data/medical-charges.csv'
from urllib.request import urlretrieve
urlretrieve(medical_charges_url, 'medical.csv')

('medical.csv', <http.client.HTTPMessage at 0x7efea1b2a590>)